# Statistical Rethinking: Excercise 2M2

Like in the previous notebook for 2M1, we have the following globe tossing results:

(1) W, W, W <br>
(2) W, W, W, L <br>
(3) L, W, W, L, W, W, W <br>

<br>
We want to estimate the probability of Water $p$ as parameter of the model. The parameter is at least 0 and at most 1.
<br>
<br>
The key point for this exercise is that we employ a different prior:
$$    \text{prior}(p) =
    \begin{cases}
      0, & \text{if}\ p < 0.5 \\
      1, & \text{otherwise}
    \end{cases}$$

In [25]:
require "rubythinking"
grid_size = 50
step_size = 1.0 / grid_size.to_f
grid = 0.step(by: step_size, to: 1).to_a

[0.0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16, 0.18, 0.2, 0.22, 0.24, 0.26, 0.28, 0.3, 0.32, 0.34, 0.36, 0.38, 0.4, 0.42, 0.44, 0.46, 0.48, 0.5, 0.52, 0.54, 0.56, 0.58, 0.6, 0.62, 0.64, 0.66, 0.68, 0.7000000000000001, 0.72, 0.74, 0.76, 0.78, 0.8, 0.8200000000000001, 0.84, 0.86, 0.88, 0.9, 0.92, 0.9400000000000001, 0.96, 0.98, 1.0]

In [42]:
prior = grid.map do |x| 
  y =
    if x < 0.5
      0
    else
      1.to_f / (grid_size.to_f / 2.0)
    end
  
  {x: x, y: y}
end

Vega.lite
  .data(prior)
  .mark(type: "area")
  .encoding(
    x: {field: "x", type: "quantitative"},
    y: {field: "y", type: "quantitative", scale: {"domain": [0,0.1]}}
  )

#<Vega::LiteChart:0x00007f9f5696ccb8 @schema="https://vega.github.io/schema/vega-lite/v4.json", @spec={"$schema"=>"https://vega.github.io/schema/vega-lite/v4.json", "width"=>"container", "height"=>"container", "data"=>{:values=>[{:x=>0.0, :y=>0}, {:x=>0.02, :y=>0}, {:x=>0.04, :y=>0}, {:x=>0.06, :y=>0}, {:x=>0.08, :y=>0}, {:x=>0.1, :y=>0}, {:x=>0.12, :y=>0}, {:x=>0.14, :y=>0}, {:x=>0.16, :y=>0}, {:x=>0.18, :y=>0}, {:x=>0.2, :y=>0}, {:x=>0.22, :y=>0}, {:x=>0.24, :y=>0}, {:x=>0.26, :y=>0}, {:x=>0.28, :y=>0}, {:x=>0.3, :y=>0}, {:x=>0.32, :y=>0}, {:x=>0.34, :y=>0}, {:x=>0.36, :y=>0}, {:x=>0.38, :y=>0}, {:x=>0.4, :y=>0}, {:x=>0.42, :y=>0}, {:x=>0.44, :y=>0}, {:x=>0.46, :y=>0}, {:x=>0.48, :y=>0}, {:x=>0.5, :y=>0.04}, {:x=>0.52, :y=>0.04}, {:x=>0.54, :y=>0.04}, {:x=>0.56, :y=>0.04}, {:x=>0.58, :y=>0.04}, {:x=>0.6, :y=>0.04}, {:x=>0.62, :y=>0.04}, {:x=>0.64, :y=>0.04}, {:x=>0.66, :y=>0.04}, {:x=>0.68, :y=>0.04}, {:x=>0.7000000000000001, :y=>0.04}, {:x=>0.72, :y=>0.04}, {:x=>0.74, :y=>0.04}, {:x=>0.76, :y=>0.04}, {:x=>0.78, :y=>0.04}, {:x=>0.8, :y=>0.04}, {:x=>0.8200000000000001, :y=>0.04}, {:x=>0.84, :y=>0.04}, {:x=>0.86, :y=>0.04}, {:x=>0.88, :y=>0.04}, {:x=>0.9, :y=>0.04}, {:x=>0.92, :y=>0.04}, {:x=>0.9400000000000001, :y=>0.04}, {:x=>0.96, :y=>0.04}, {:x=>0.98, :y=>0.04}, {:x=>1.0, :y=>0.04}]}, "mark"=>{:type=>"area"}, "encoding"=>{:x=>{:field=>"x", :type=>"quantitative"}, :y=>{:field=>"y", :type=>"quantitative", :scale=>{:domain=>[0, 0.1]}}}}>

In [38]:
factorial = ->(n) do
  return 1 if n < 1
  n.to_i.downto(1).inject(:*)
end


likelihood = ->(w, l, p) do
  (factorial[w+l].to_f / (factorial[w] * factorial[l])).to_f * (p**w) * ((1-p)**l)
end

#<Proc:0x00007f9f558c8d08@<main>:6 (lambda)>

Now, let's compute the grid aprroximation of the posterior for each of the cases. The difference is only the data input we give in terms of "count of Water" versus "count of Land" of our tossing result given in the exercise.

In [45]:
# For case (1)
w = 3
l = 0

print_posterior = ->(w, l) do
  u_posterior = grid.map do |x| 
    prior_x = prior.detect { |p| p[:x] == x }[:y]
    y = prior_x * likelihood[w, l, x]
    {x: x, y: y}
  end

  posterior = u_posterior.map do |pair| 
    y = pair[:y].to_f / u_posterior.map { |pair| pair[:y] }.sum.to_f
   {x: pair[:x], y: y}
  end

  Vega.lite
    .data(posterior)
    .mark(type: "area")
    .encoding(
      x: {field: "x", type: "quantitative"},
      y: {field: "y", type: "quantitative", scale: {"domain": [0,0.1]}}
    )
end

print_posterior[3, 0]

#<Vega::LiteChart:0x00007f9f569460b8 @schema="https://vega.github.io/schema/vega-lite/v4.json", @spec={"$schema"=>"https://vega.github.io/schema/vega-lite/v4.json", "width"=>"container", "height"=>"container", "data"=>{:values=>[{:x=>0.0, :y=>0.0}, {:x=>0.02, :y=>0.0}, {:x=>0.04, :y=>0.0}, {:x=>0.06, :y=>0.0}, {:x=>0.08, :y=>0.0}, {:x=>0.1, :y=>0.0}, {:x=>0.12, :y=>0.0}, {:x=>0.14, :y=>0.0}, {:x=>0.16, :y=>0.0}, {:x=>0.18, :y=>0.0}, {:x=>0.2, :y=>0.0}, {:x=>0.22, :y=>0.0}, {:x=>0.24, :y=>0.0}, {:x=>0.26, :y=>0.0}, {:x=>0.28, :y=>0.0}, {:x=>0.3, :y=>0.0}, {:x=>0.32, :y=>0.0}, {:x=>0.34, :y=>0.0}, {:x=>0.36, :y=>0.0}, {:x=>0.38, :y=>0.0}, {:x=>0.4, :y=>0.0}, {:x=>0.42, :y=>0.0}, {:x=>0.44, :y=>0.0}, {:x=>0.46, :y=>0.0}, {:x=>0.48, :y=>0.0}, {:x=>0.5, :y=>0.010175010175010175}, {:x=>0.52, :y=>0.011445502645502646}, {:x=>0.54, :y=>0.01281758241758242}, {:x=>0.56, :y=>0.0142951566951567}, {:x=>0.58, :y=>0.015882132682132677}, {:x=>0.6, :y=>0.01758241758241758}, {:x=>0.62, :y=>0.0193999185999186}, {:x=>0.64, :y=>0.021338542938542943}, {:x=>0.66, :y=>0.023402197802197804}, {:x=>0.68, :y=>0.025594790394790397}, {:x=>0.7000000000000001, :y=>0.027920227920227927}, {:x=>0.72, :y=>0.03038241758241758}, {:x=>0.74, :y=>0.03298526658526658}, {:x=>0.76, :y=>0.03573268213268214}, {:x=>0.78, :y=>0.03862857142857143}, {:x=>0.8, :y=>0.041676841676841686}, {:x=>0.8200000000000001, :y=>0.04488140008140009}, {:x=>0.84, :y=>0.048246153846153834}, {:x=>0.86, :y=>0.05177501017501017}, {:x=>0.88, :y=>0.05547187627187627}, {:x=>0.9, :y=>0.05934065934065935}, {:x=>0.92, :y=>0.06338526658526658}, {:x=>0.9400000000000001, :y=>0.06760960520960523}, {:x=>0.96, :y=>0.07201758241758241}, {:x=>0.98, :y=>0.0766131054131054}, {:x=>1.0, :y=>0.0814000814000814}]}, "mark"=>{:type=>"area"}, "encoding"=>{:x=>{:field=>"x", :type=>"quantitative"}, :y=>{:field=>"y", :type=>"quantitative", :scale=>{:domain=>[0, 0.1]}}}}>

In [46]:
# For case (2)
w = 3
l = 1
print_posterior[3, 1]

#<Vega::LiteChart:0x00007f9f54198e08 @schema="https://vega.github.io/schema/vega-lite/v4.json", @spec={"$schema"=>"https://vega.github.io/schema/vega-lite/v4.json", "width"=>"container", "height"=>"container", "data"=>{:values=>[{:x=>0.0, :y=>0.0}, {:x=>0.02, :y=>0.0}, {:x=>0.04, :y=>0.0}, {:x=>0.06, :y=>0.0}, {:x=>0.08, :y=>0.0}, {:x=>0.1, :y=>0.0}, {:x=>0.12, :y=>0.0}, {:x=>0.14, :y=>0.0}, {:x=>0.16, :y=>0.0}, {:x=>0.18, :y=>0.0}, {:x=>0.2, :y=>0.0}, {:x=>0.22, :y=>0.0}, {:x=>0.24, :y=>0.0}, {:x=>0.26, :y=>0.0}, {:x=>0.28, :y=>0.0}, {:x=>0.3, :y=>0.0}, {:x=>0.32, :y=>0.0}, {:x=>0.34, :y=>0.0}, {:x=>0.36, :y=>0.0}, {:x=>0.38, :y=>0.0}, {:x=>0.4, :y=>0.0}, {:x=>0.42, :y=>0.0}, {:x=>0.44, :y=>0.0}, {:x=>0.46, :y=>0.0}, {:x=>0.48, :y=>0.0}, {:x=>0.5, :y=>0.030333668411168073}, {:x=>0.52, :y=>0.032756401520313755}, {:x=>0.54, :y=>0.03515475121344381}, {:x=>0.56, :y=>0.03750262568233768}, {:x=>0.58, :y=>0.03977206941818762}, {:x=>0.6, :y=>0.041933263211598744}, {:x=>0.62, :y=>0.043954524152588936}, {:x=>0.64, :y=>0.04580230563058892}, {:x=>0.66, :y=>0.047441197334442235}, {:x=>0.68, :y=>0.04883392525240524}, {:x=>0.7000000000000001, :y=>0.04994135167214711}, {:x=>0.72, :y=>0.05072247518074984}, {:x=>0.74, :y=>0.05113443066470823}, {:x=>0.76, :y=>0.05113248930992992}, {:x=>0.78, :y=>0.05067005860173534}, {:x=>0.8, :y=>0.04969868232485778}, {:x=>0.8200000000000001, :y=>0.04816804056344327}, {:x=>0.84, :y=>0.04602594970105078}, {:x=>0.86, :y=>0.04321836242065198}, {:x=>0.88, :y=>0.03968936770463141}, {:x=>0.9, :y=>0.035381190834786434}, {:x=>0.92, :y=>0.030234193392327212}, {:x=>0.9400000000000001, :y=>0.024186873257876735}, {:x=>0.96, :y=>0.01717586461147086}, {:x=>0.98, :y=>0.00913593793255812}, {:x=>1.0, :y=>0.0}]}, "mark"=>{:type=>"area"}, "encoding"=>{:x=>{:field=>"x", :type=>"quantitative"}, :y=>{:field=>"y", :type=>"quantitative", :scale=>{:domain=>[0, 0.1]}}}}>

In [47]:
# For case (3)
w = 5
l = 2
print_posterior[5, 2]

#<Vega::LiteChart:0x00007f9f540bb738 @schema="https://vega.github.io/schema/vega-lite/v4.json", @spec={"$schema"=>"https://vega.github.io/schema/vega-lite/v4.json", "width"=>"container", "height"=>"container", "data"=>{:values=>[{:x=>0.0, :y=>0.0}, {:x=>0.02, :y=>0.0}, {:x=>0.04, :y=>0.0}, {:x=>0.06, :y=>0.0}, {:x=>0.08, :y=>0.0}, {:x=>0.1, :y=>0.0}, {:x=>0.12, :y=>0.0}, {:x=>0.14, :y=>0.0}, {:x=>0.16, :y=>0.0}, {:x=>0.18, :y=>0.0}, {:x=>0.2, :y=>0.0}, {:x=>0.22, :y=>0.0}, {:x=>0.24, :y=>0.0}, {:x=>0.26, :y=>0.0}, {:x=>0.28, :y=>0.0}, {:x=>0.3, :y=>0.0}, {:x=>0.32, :y=>0.0}, {:x=>0.34, :y=>0.0}, {:x=>0.36, :y=>0.0}, {:x=>0.38, :y=>0.0}, {:x=>0.4, :y=>0.0}, {:x=>0.42, :y=>0.0}, {:x=>0.44, :y=>0.0}, {:x=>0.46, :y=>0.0}, {:x=>0.48, :y=>0.0}, {:x=>0.5, :y=>0.030230440239998296}, {:x=>0.52, :y=>0.03389640455469191}, {:x=>0.54, :y=>0.03759576306122878}, {:x=>0.56, :y=>0.041257217006775745}, {:x=>0.58, :y=>0.0448015448911555}, {:x=>0.6, :y=>0.04814272579711524}, {:x=>0.62, :y=>0.051189405068867475}, {:x=>0.64, :y=>0.05384672730179831}, {:x=>0.66, :y=>0.05601856160623748}, {:x=>0.68, :y=>0.05761014410818551}, {:x=>0.7000000000000001, :y=>0.058531162649892644}, {:x=>0.72, :y=>0.05869930865318412}, {:x=>0.74, :y=>0.05804432110842732}, {:x=>0.76, :y=>0.05651254765203498}, {:x=>0.78, :y=>0.054072047695399646}, {:x=>0.8, :y=>0.05071826256815431}, {:x=>0.8200000000000001, :y=>0.04648027763865389}, {:x=>0.84, :y=>0.04142770137457235}, {:x=>0.86, :y=>0.03567818630651054}, {:x=>0.88, :y=>0.02940561685750976}, {:x=>0.9, :y=>0.022848989001365234}, {:x=>0.92, :y=>0.016322006712635093}, {:x=>0.9400000000000001, :y=>0.010223420171239313}, {:x=>0.96, :y=>0.005048130684543599}, {:x=>0.98, :y=>0.0013990872898229294}, {:x=>1.0, :y=>0.0}]}, "mark"=>{:type=>"area"}, "encoding"=>{:x=>{:field=>"x", :type=>"quantitative"}, :y=>{:field=>"y", :type=>"quantitative", :scale=>{:domain=>[0, 0.1]}}}}>